In [58]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt


In [59]:
dataset_dir = 'C:/Users/sreevishak/Desktop/DUK/smart india/archive/'  # Adjust the path as needed


In [60]:
# Data augmentation and preprocessing for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Use 20% of data for validation
)

# Create training and validation data generators
train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'  # Use the training subset
)

val_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Use the validation subset
)


Found 40287 images belonging to 23 classes.
Found 10059 images belonging to 23 classes.


In [61]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


C:\Users\sreevishak\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [62]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

history = model.fit(
    train_generator,
    epochs=30,
    validation_data=val_generator,
    callbacks=[early_stopping]
)


Epoch 1/30


C:\Users\sreevishak\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


   4/1259 ━━━━━━━━━━━━━━━━━━━━ 13:44 657ms/step - accuracy: 0.3229 - loss: 2.9864

UnknownError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x0000023AD251FB50>
Traceback (most recent call last):

  File "C:\Users\sreevishak\AppData\Roaming\Python\Python311\site-packages\tensorflow\python\ops\script_ops.py", line 270, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "C:\Users\sreevishak\AppData\Roaming\Python\Python311\site-packages\tensorflow\python\autograph\impl\api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "C:\Users\sreevishak\AppData\Roaming\Python\Python311\site-packages\tensorflow\python\data\ops\from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "C:\Users\sreevishak\AppData\Roaming\Python\Python311\site-packages\keras\

In [47]:
"""
    Encode the target label
    
    @param label - the target label to be encoded
    @return a encoded label
"""
def encode_label(label):
  if label == 'Cashew':
    return 0
  elif label == 'Cassava':
    return 1
  elif label == 'Maize':
    return 2
  elif label == 'Tomato':
    return 3
  else:
    return -1

In [48]:
filenames = []
plant_category = []
disease_category = []

# sort the categories and filenames into lists
for (root, dirs, files) in os.walk(DATASET_DIR):
  for filename in files:
    fn = str(root + "/" + filename)
    filenames.append(fn)

    pc = get_plant_category_by_path(fn)
    plant_category.append(pc)

    dc = get_disease_category_by_path(fn)
    disease_category.append(dc)

In [49]:
# create a Pandas dataframe using the lists
data = { "filename": filenames, "plant category": plant_category, "disease category": disease_category }
df = pd.DataFrame(data)
df = df.sample(frac=1)  # shuffle the dataframe

print(df.head(10))

                                                filename plant category  \
1847   C:/Users/sreevishak/Desktop/DUK/smart india/ar...            DUK   
7      C:/Users/sreevishak/Desktop/DUK/smart india/ar...            DUK   
22334  C:/Users/sreevishak/Desktop/DUK/smart india/ar...            DUK   
10086  C:/Users/sreevishak/Desktop/DUK/smart india/ar...            DUK   
5779   C:/Users/sreevishak/Desktop/DUK/smart india/ar...            DUK   
18184  C:/Users/sreevishak/Desktop/DUK/smart india/ar...            DUK   
14548  C:/Users/sreevishak/Desktop/DUK/smart india/ar...            DUK   
1054   C:/Users/sreevishak/Desktop/DUK/smart india/ar...            DUK   
14363  C:/Users/sreevishak/Desktop/DUK/smart india/ar...            DUK   
16137  C:/Users/sreevishak/Desktop/DUK/smart india/ar...            DUK   

      disease category  
1847               DUK  
7                  DUK  
22334              DUK  
10086              DUK  
5779               DUK  
18184              DUK  

In [50]:
# remove invalid images from the dataframe
df = clean_image_dataframe(df)
print(f"Length of DF: {len(df['filename'])}")

Length of DF: 25124


In [51]:
category_sizes = df.groupby("plant category").size()
print(category_sizes)

plant category
DUK    25124
dtype: int64


In [53]:
# truncate the number of images in each category so each have an equal amount
cashew_files = [df['filename'].values[index] for index, x in enumerate(df['plant category'].values) if 'Cashew' == x][0:MAX_IMAGES_PER_CATEGORY]
cassava_files = [df['filename'].values[index] for index, x in enumerate(df['plant category'].values) if 'Cassava' == x][0:MAX_IMAGES_PER_CATEGORY]
maize_files = [df['filename'].values[index] for index, x in enumerate(df['plant category'].values) if 'Maize' == x][0:MAX_IMAGES_PER_CATEGORY]
tomato_files = [df['filename'].values[index] for index, x in enumerate(df['plant category'].values) if 'Tomato' == x][0:MAX_IMAGES_PER_CATEGORY]

dataset = []

def create_dataset(filenames, label):
  for filename in filenames:
    data_subset = []
    data_subset.append(filename)
    data_subset.append(encode_label(label))

    dataset.append(data_subset)

create_dataset(cashew_files, 'Cashew')
create_dataset(cassava_files, 'Cassava')
create_dataset(maize_files, 'Maize')
create_dataset(tomato_files, 'Tomato')

unbalanced_dataset_size = len(df['filename'].unique())
balanced_dataset_size = len(dataset)
loss = round((1 - (balanced_dataset_size / unbalanced_dataset_size)) * 100, 2)

print(f"Size of Unbalanced Dataset: {unbalanced_dataset_size}")
print(f"Size of Balanced Dataset: {balanced_dataset_size}")

Size of Unbalanced Dataset: 25124
Size of Balanced Dataset: 0


In [54]:
dataset_df = pd.DataFrame(dataset, columns=['filename', 'label'])
print(dataset_df.head())

Empty DataFrame
Columns: [filename, label]
Index: []


In [57]:
X = dataset_df['filename']
y = dataset_df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

ValueError: With n_samples=0, test_size=0.25 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.